In [17]:
# please write your player id
player_id = '1'

In [18]:
import os 

PATH = os.path.join(os.getcwd(),'kidney-game')

p_PATH = os.path.join(PATH,'player'+player_id)

git_dir = "https://github.com/jjbl99/kidney-game.git" # address of your repo on git

import importlib
import sys 
# sys.path.append(PATH)
sys.path.append('./kidney-game')

import git
from git import Repo

if not os.path.exists(PATH):
    Repo.clone_from(git_dir, PATH)

import PUBLIC_market
importlib.reload(PUBLIC_market)
from PUBLIC_market import *

repo = Repo(PATH) # create an object "repo" that corresponds to your local repository
PUBLIC_market.init(repo, PATH)

# Optimization Functions 
### (Coded by the student)

In [19]:
import gurobipy as gp
from gurobipy import GRB

ModuleNotFoundError: No module named 'gurobipy'

In [20]:
# What the student would have to code

def intlinprogGurobi(w,A,b,lb,ub,N,mute=1): 
    #misses Aeq, beq, options, and intcon from original matlab code 
    m = gp.Model("model")
    x = m.addMVar(shape=N, lb=lb, ub=ub, vtype=GRB.BINARY, name="x")
    if mute: 
        m.setParam('OutputFlag', 0)
    # Set objective
    m.setObjective(w @ x, GRB.MINIMIZE)
    
    # add constraints 
    m.addConstr((A @ x) <= b, name="c")
    
    # Optimize model
    m.optimize()
    match = x.X

    return match.astype(int)

def f_weights(G_coord):
    return - np.ones(G_coord.shape[0])

def recursive_KEP(G_coord,G,n,K,L,M,NDD): 
    """
    Recursive KEP algorithm from Anderson, Ashlagi et al (2014)
    Function that returns the optimal kidney exchange for a given compatibility graph
    
    G: n*n adjacency matrix where G(i,j) means that donor i can give to patient j
    K: max length of cycles to use
    L: max length of chains to use 
    M: max number of chains to use
    f_weights: function that takes G as an argument and returns a matrix w of weights of size n*n
    NDD: list of indexes of donors that are connected to a NDD
    """
    
    ## Initialization
    k = NDD.size # number of NDDs
    e = G_coord.shape[0]
    N = e + k # number of variables to study
    
    ## plot initial graph 
    # show_graph(G,NDD)
    
    # Complete the coordinate representation of my graph
    if k > 0:
        G_coord = np.concatenate((G_coord, np.concatenate((np.arange(n,n+k)[:,np.newaxis], NDD[:,np.newaxis]),axis=1)), axis = 0) # we add the NDDs here
    ## Construct constraint matrices A and b
    Minf = int(M != inf)
    A = np.zeros((2*n+k+Minf,N))
    r1 = np.concatenate((G_coord[:e,1], n+G_coord[:e,0], n+G_coord[0:e,1]), axis = 0) 
    c1 = np.repeat(np.arange(0,e)[:,np.newaxis],3,axis =1).T.reshape(-1)
    A[r1, c1] = np.concatenate((np.ones(2*e), -np.ones(e)))
    
    A[G_coord[e:N,1],np.arange(e,N)] = 1 # need to make sure that no NDD goes into a cycle
    A[n+G_coord[e:N,1], np.arange(e,N)] = -1
    # Add constraint that flow out of a NDD is less than the flow out the pair he gives to 
    A[np.arange(2*n,2*n+k), np.arange(e,N)] = 1
    positions = (np.repeat(NDD[:,np.newaxis],e,axis=1).T == np.repeat(G_coord[:e,0,np.newaxis],k,axis =1)).T.astype(int) # matrix of size k*e corresponding to our constraints
    A[(2*n):(2*n+k),:e] = - positions
    if Minf:
        A[-1,e:] = 1; # used for the constraint on number of chains
        b = np.ones(2*n+k+1)
        b[-1] = M
        b[n:2*n+k] = 0
    else:
        b = np.ones(2*n+k)
        b[n:2*n+k] = 0
    
    ## Initial matching without constraints on chain and cycle sizes
    lb, ub =np.zeros(N), np.ones(N)
    w = f_weights(G_coord) # weights
    
    match = intlinprogGurobi(w,A,b,lb,ub,N)
    
    ## Re-optimize until all chains and cycles have the desired size
    pb = 1
    ii = 0

    while pb:
        cycles, chains = check_match(match.copy(),G_coord,n,e,k)
        sizeCy = np.sum(cycles, axis = 1) if cycles.size > 0 else np.array([])
        sizeCh = np.sum(chains, axis = 1) if chains.size > 0 else np.array([])
        maxCh = max(np.max(sizeCh),0) if sizeCh.size > 0 else 0
        maxCy = max(np.max(sizeCy),0) if sizeCy.size > 0 else 0
        if ((maxCh <= L) and (maxCy <= K)):
            pb = 0
        else:
            idCy = sizeCy > (np.ones(sizeCy.shape)*K)
            idCh = sizeCh > (np.ones(sizeCh.shape)*L)
            
            if (cycles.size != 0) and (chains.size != 0):
                A = np.concatenate((A, cycles[idCy,:], chains[idCh,:]), axis = 0) # add constraint to eliminate chains/cycles that are too big                  
                b = np.concatenate((b, K * np.ones(np.sum(idCy)), L * np.ones(np.sum(idCh))), axis = 0) 
            elif cycles.size != 0: 
                A = np.concatenate((A, cycles[idCy,:]), axis = 0) # add constraint to eliminate chains/cycles that are too big                  
                b = np.concatenate((b, K * np.ones(np.sum(idCy))), axis = 0)
            elif chains.size != 0:
                A = np.concatenate((A, chains[idCh,:]), axis = 0) # add constraint to eliminate chains/cycles that are too big                  
                b = np.concatenate((b, L * np.ones(np.sum(idCh))), axis = 0) 
                
            match = intlinprogGurobi(w,A,b,lb,ub,N)
            ii += 1

    return match # cycles,chains

## Playing the game

In [21]:
# pull any changes
git_pull()

In [22]:
# initialize game
new_game(p_PATH)

In [7]:
# demo of the attributes and function of the class Market
# TO DO 
# market.plot_graph()
# market.comp_stats()

In [8]:
# git_pull()
# market = get_market(p_PATH)
# make class Market and Market_nature

In [23]:
frequency = 2
T = 15

for t in range(T):
    git_pull()
    file = open(os.path.join(p_PATH,'status.txt'),"r")
    instruction = next(file)
    file.close()
    
    while str(instruction.split()[0]) != "nature": # TRY AND READ DIRECTLY 
        # wait until nature played
        git_pull()
        file = open(os.path.join(p_PATH,'status.txt'),"r")
        instruction = next(file)
        file.close()
    
    print('youre in')
    print(t)
    market = get_market(p_PATH) # function coded in PUBLIC_market.py where all functions are
    
    if t % frequency == 0:
        G_coord = adj_to_coord(market.graph)
        match = recursive_KEP(G_coord,market.graph,market.n,market.K,market.L,market.M,market.NDDs)

        submit(p_PATH, t, match) # If match == [], go to next step, else pairs in match are taken away
    else:
        submit(p_PATH, t)

youre in
0
youre in
1
youre in
2
youre in
3
youre in
4


GitCommandError: Cmd('git') failed due to: exit code(1)
  cmdline: git pull -v origin
  stderr: 'fatal: unable to access 'https://github.com/jjbl99/kidney-game.git/': Failed to connect to github.com port 443: Operation timed out'

In [ ]:
git_pull()
# function status that pulls, and: 
    # prints status.txt, tells you how many period left you have to play
    # if t == T, prints : "you finished playing, to see measures of your performance, call the function market.stats()"
    # "to start a new game, use the function new_game(). If you are satisfied with your submission and want to submit it 
    # as your final submission use end_game(). CAREFUL: you will not be able to play after that"

In [ ]:
# file = open(p_PATH+'/status.txt',"r")
# instruction = next(file)
# instruction.split()

In [ ]:
# new_game()

In [ ]:
# this function let the nature knows that this is your final submission. CAREFUL: YOU CANNOT GO BACK
# end_game()

# To do still 
- Function that computes statistics automatically 
- improve exit rate 
- what should be black box (easier: what shouldn't be? K,L,M, initial size etc. Should be black box: arrival and exit processes). Process to decide whether patients are matched or not can (should?) be black box. Do we want to help them by avoiding cold start (i.e. by giving them P_stats?). 
- How can we make the game dynamic? Need to be able to communicate with a server where the info is stored. 

# Idea of the game 

You have an initial market. 
At each period, the market gets updated (new arrival, deaths). 

You (the player) can choose to match some people or not. To do so you simply do m.market_update(match) where match is a vector of 0s and 1s of size corresponding to total number of edges (including to NDDs). CAREFUL: it is important that the match vector you submit coincides with the G_coord obtained through adj_to_coord().

GOAL: in a T period game, match as many patients as possible. 


ADD SELF.T TO MARKET AND TO MARKET2CSV